In [0]:
!pip install numpy==1.26.4
!pip install itables
!pip install itables[widget]
!pip install chainladder
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import numpy as np
import pandas as pd
import chainladder as cl
import warnings

In [0]:
#extract weights in a single column
def get_weights(dev):
    if hasattr(dev, "w_"):
        weights = np.reshape(dev.w_[...,:-1,:],(-1))
        weights = np.nan_to_num(weights)
        weights = np.argwhere(weights==1)
        weights = np.reshape(weights,(-1))
        return weights
    else:
        warnings.warn("Development has not been fitted.")
        return None

In [0]:
#extract factors in a single column
def get_factors(tri):
    return tri.link_ratio.to_frame().melt(var_name='DevPeriod', value_name='LDF',ignore_index=False)

In [0]:
#convert selected weights back into an argument for the development object
def get_drop(od, sel):
    unsel = list(set(range(len(df))) - set(sel))
    val = od.iloc[unsel].reset_index()
    val['index'] = val['index'].astype(str)
    val['DevPeriod'] = val['DevPeriod'].str.extract('^([^_]*)-')
    val['DevPeriod'] = val['DevPeriod'].astype(int)
    val = val[['index','DevPeriod']].itertuples(index=False,name=None)
    return list(val)

In [0]:
from itables.widget import ITable
genins = cl.load_sample("genins")
#drop_high=0 is not purely decorational. Specifying n_periods only yields an incorrect but non-consequential weight array in that there is an extra diagonal of 1.
dev = cl.Development(n_periods=4, drop_high=0).fit_transform(genins)
w = get_weights(dev)
df = get_factors(genins)
table = ITable(df, selected_rows=w.tolist(), select="multi", lengthMenu = [9])
table

ITable(classes='display nowrap', selected_rows=[5, 6, 7, 8, 13, 14, 15, 16, 21, 22, 23, 24, 29, 30, 31, 32, 37…

In [0]:
val = get_drop(df, table.selected_rows)
manual_dev = cl.Development(drop = val).fit(genins)
print("LDF from original selection")
print(dev.ldf_)
print("LDF from UI selection")
print(manual_dev.ldf_)

LDF from original selection
          12-24    24-36     36-48     48-60    60-72     72-84     84-96    96-108   108-120
(All)  3.436526  1.85231  1.470665  1.173507  1.08481  1.086269  1.053874  1.076555  1.017725
LDF from UI selection
          12-24     24-36     36-48     48-60    60-72     72-84     84-96    96-108   108-120
(All)  3.243668  1.927407  1.470665  1.173507  1.08481  1.086269  1.053874  1.076555  1.017725
